In [7]:
import pandas as pd
import numpy as np
from typing import List
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
# Importe a classe de clustering hierárquico
from sklearn.cluster import AgglomerativeClustering
# Importe NearestNeighbors para atribuir pontos de teste
from sklearn.neighbors import NearestNeighbors

import mlflow
# Certifique-se de que a biblioteca static_models está instalada ou disponível
from model2vec import StaticModel # Assumindo que StaticModel é como no seu código
# Assumindo que Embeddings é uma classe base que você definiu ou importou
from langchain.embeddings.base import Embeddings # Assumindo que é de langchain ou similar

In [8]:
mlflow.set_tracking_uri("http://127.0.0.1:5000/")  # Ajuste para seu servidor MLflow
mlflow.set_experiment("Train_Emb_Balanced_Synth-Negative_Sentiment_Analysis_Restaurant")

<Experiment: artifact_location='mlflow-artifacts:/816380000824485392', creation_time=1747411527299, experiment_id='816380000824485392', last_update_time=1747411527299, lifecycle_stage='active', name='Train_Emb_Balanced_Synth-Negative_Sentiment_Analysis_Restaurant', tags={}>

INPUTS

In [9]:
# --- Carregar Dados ---
# Use um caminho relativo ou absoluto correto para o seu arquivo
# Exemplo:
# data_path = '../data/dataset_train_with_sentiment_fix_negative_trimmed_similarity.parquet'
# df_train = pd.read_parquet(data_path)

# Usando o caminho do seu código
df_train = pd.read_parquet('../data\dataset_train_trim_synthetic_balanced.parquet')


In [10]:
if 'target' not in df_train.columns:
    df_train['target'] = df_train['sentiment']

X_text = df_train['comment_cleaned']
y = df_train['target']

# --- Split de Treino e Teste ---
print("Realizando split de treino/teste...")
X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text, y, test_size=0.2, random_state=42, stratify=y
)

# --- Sentence Embedding ---
class Model2VecEmbeddings(Embeddings):
    """Wrapper para o Model2Vec como Embeddings do LangChain"""
    def __init__(self, model_name: str):
        # Certifique-se de ter a biblioteca static_models instalada (pip install static-models)
        self.model = StaticModel.from_pretrained(model_name)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        # Certifique-se de que texts é uma lista de strings
        return self.model.encode(texts).tolist()

    def embed_query(self, text: str) -> List[float]:
        # Certifique-se de que text é uma string
        return self.model.encode([text]).tolist()[0]

print("Gerando embeddings com Model2Vec...")
model_name = "minishlab/potion-base-8M"
# Instancie a classe e use para gerar embeddings
model = Model2VecEmbeddings(model_name)

# Converta as Series de texto para listas antes de passar para o modelo
X_train_vec = model.embed_documents(X_train_text.tolist())
X_test_vec = model.embed_documents(X_test_text.tolist())

# Converta as listas de embeddings para arrays numpy para facilitar o processamento
X_train_vec = np.array(X_train_vec)
X_test_vec = np.array(X_test_vec)


# --- Clustering (Hierarchical) ---
print("Executando Hierarchical Clustering...")

n_clusters = 3 # Manter o mesmo número de clusters

# Pode escolher diferentes linkages ('ward', 'complete', 'average', 'single')
# 'ward' é comum para embeddings baseados em distância Euclidiana e minimiza a variância dentro dos clusters
hierarchical_clustering = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')

# Ajustar o modelo nos dados de treino e obter os labels
train_clusters = hierarchical_clustering.fit_predict(X_train_vec)

# --- Atribuir clusters para os dados de teste ---
# Como AgglomerativeClustering não tem 'predict',
# atribuímos cada ponto de teste ao cluster do ponto de treino mais próximo.

print(f"Atribuindo clusters do teste usando o vizinho mais próximo dos dados de treino ({n_clusters} clusters)...")
# Crie um modelo de vizinhos mais próximos nos dados de treino
nn = NearestNeighbors(n_neighbors=1, metric='euclidean') # Encontre o único vizinho mais próximo
nn.fit(X_train_vec)

# Para cada ponto no conjunto de teste, encontre o índice do seu vizinho mais próximo no treino
distances, indices = nn.kneighbors(X_test_vec) # indices terá shape (n_samples_test, 1)

# Use os índices encontrados para obter os rótulos de cluster correspondentes do treino
# .flatten() transforma o array de índices de (n_samples_test, 1) para (n_samples_test,)
test_clusters = train_clusters[indices.flatten()]


# Adicionar clusters como feature extra
# Certifique-se que os arrays de clusters têm a forma correta (n_samples, 1)
X_train_augmented = np.hstack([X_train_vec, train_clusters.reshape(-1, 1)])
X_test_augmented = np.hstack([X_test_vec, test_clusters.reshape(-1, 1)])

# --- Modelo Supervisionado (Logistic Regression) ---
model_name = "LogReg_with_Embeddings+Hierarchical"

with mlflow.start_run(run_name=model_name):
    print(f"Treinando modelo supervisionado: {model_name}...")
    # Ajuste os parâmetros da Regressão Logística conforme necessário
    clf = LogisticRegression(C=10, penalty='l2', solver='lbfgs', max_iter=1000, random_state=42)
    clf.fit(X_train_augmented, y_train)

    print("Avaliando modelo...")
    y_pred = clf.predict(X_test_augmented)
    acc = accuracy_score(y_test, y_pred)
    # Gere o relatório de classificação como um dicionário para MLflow
    report = classification_report(y_test, y_pred, output_dict=True)
    # Gere o relatório de classificação para exibição no console
    report_str = classification_report(y_test, y_pred)


    print(f"Accuracy: {acc:.4f}")
    print("Classification Report:")
    print(report_str)

    # --- Log MLflow ---
    # Não use autolog() se você está registrando parâmetros e métricas manualmente
    # mlflow.autolog() # Remova ou comente esta linha se registrar manualmente

    mlflow.log_param("embedding_model", model_name) # O nome do modelo de embedding
    mlflow.log_param("classifier", type(clf).__name__) # Nome da classe do classificador
    mlflow.log_param("C", clf.C)
    mlflow.log_param("penalty", clf.penalty)
    mlflow.log_param("solver", clf.solver)
    mlflow.log_param("max_iter", clf.max_iter)
    # Logar os parâmetros do clustering hierárquico
    mlflow.log_param("clustering_method", "Hierarchical Clustering")
    mlflow.log_param("hierarchical_clusters", n_clusters)
    mlflow.log_param("hierarchical_linkage", hierarchical_clustering.linkage)
    mlflow.log_param("test_cluster_assignment", "NearestNeighbor") # Método usado para teste

    mlflow.log_metric("accuracy", acc)
    mlflow.log_dict(report, "classification_report.json")

print("Pipeline concluído.")

Realizando split de treino/teste...
Gerando embeddings com Model2Vec...


model.safetensors:   0%|          | 0.00/30.2M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/271k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/202 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/684k [00:00<?, ?B/s]

Executando Hierarchical Clustering...
Atribuindo clusters do teste usando o vizinho mais próximo dos dados de treino (3 clusters)...
Treinando modelo supervisionado: LogReg_with_Embeddings+Hierarchical...
Avaliando modelo...
Accuracy: 0.7007
Classification Report:
              precision    recall  f1-score   support

    Negative       0.74      0.62      0.67        52
     Neutral       0.62      0.80      0.70        35
    Positive       0.73      0.72      0.72        60

    accuracy                           0.70       147
   macro avg       0.70      0.71      0.70       147
weighted avg       0.71      0.70      0.70       147

🏃 View run LogReg_with_Embeddings+Hierarchical at: http://127.0.0.1:5000/#/experiments/816380000824485392/runs/f0fc65abdcfd4738a46928f558078be8
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/816380000824485392
Pipeline concluído.
